In [37]:
import re
import nltk
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import gutenberg
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# semantics = the meaning of a word
# Latent Semantic Analysis = 

In [28]:
# term-document matrix; columns=sentences; rows=words; 
# ignore stopwords; lemmatize

In [29]:
# term-document can be chunks, sentences, paragraphs, text files

In [30]:
# Document frequency = sentences a word appears
# Collection frequency = how often word appears
# Inverse Document Frequency = log(Total Docs / Df)
#           idf: tell to consider best as less important than others
# tf-idf = weighted based on word frequency

$$idf_t=log \dfrac N{df_t}$$

$$tf-idf_{t,d}=(tf_{t,d})(idf_t)$$


In [31]:
print('4:', 1 * np.log2(6/2), 1 * np.log2(6/3), 0, 1 * np.log2(6/3), 1 * np.log2(6/2), 0, 0, 0, 0)
print('5:', 0, 0, 0, 0, 0, 1 * np.log2(6/4), 1 * np.log2(6/3), 1* np.log2(6/2), 1*np.log2(6/3))
print('6:', 0, 0, 0, 0, 0, 0, 1*np.log2(6/3), 0, 2*np.log2(6/3))

4: 1.584962500721156 1.0 0 1.0 1.584962500721156 0 0 0 0
5: 0 0 0 0 0 0.5849625007211562 1.0 1.584962500721156 1.0
6: 0 0 0 0 0 0 1.0 0 2.0


In [38]:
gutenberg.paras
emma=gutenberg.paras('austen-persuasion.txt')
emma_paras=[]
for paragraph in emma: # clean austen text
    para=paragraph[0]
    para=[re.sub(r'--','',word) for word in para]
    emma_paras.append(' '.join(para))

In [33]:
X_train, X_test = train_test_split(emma_paras, test_size=0.4, random_state=0)
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', lowercase=True, use_idf=True,norm=u'l2',smooth_idf=True )
emma_tfidf=vectorizer.fit_transform(emma_paras)
print('Number of features:', emma_s_tfidf.get_shape()[1])

X_train_tfidf, X_test_tfidf= train_test_split(emma_tfidf, test_size=0.4, random_state=0)
X_train_tfidf_csr = X_train_tfidf.tocsr()
n = X_train_tfidf_csr.shape[0]
tfidf_bypara = [{} for _ in range(0,n)]
terms = vectorizer.get_feature_names()
for i, j in zip(*X_train_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_train_tfidf_csr[i, j]
print('Original sentence:', X_train[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 1232
Original sentence: " My expressions startle you .
Tf_idf vector: {'expressions': 1.0}


In [40]:
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
X_train_lsa = lsa.fit_transform(X_train_tfidf)
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("% variance captured by components:",total_variance*100)
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
# for i in range(5):
#     print('Component {}:'.format(i))
#     print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

% variance captured by components: 59.14611185886678


In [42]:
print('Original:', X_train[4])
print('Tf_idf:', tfidf_bypara[4])

Original: Captain Harville smiled , as much as to say , " Do you claim that for your sex ?"
Tf_idf: {'smiled': 0.5295823972120356, 'harville': 0.428412842064262, 'captain': 0.28306745311568243, 'say': 0.4010114352525943, 'claim': 0.5432012215568285}


In [ ]:
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
X_train_lsa = lsa.fit_transform(X_train_tfidf)
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])